In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
# using first-order-model for deepfake models from https://github.com/AliaksandrSiarohin/first-order-model

!pip install ffmpeg-python

https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw

copy 'vox-cpk.pth.tar' file to your drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd first-order-model

In [5]:
# import some libraries

import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

import ffmpeg

In [6]:
# input image 
source_image = imageio.imread('/content/gdrive/My Drive/Colab Notebooks/Test/Data/image.jpg')  # your image dir

# input video
video_source = imageio.mimread('/content/gdrive/My Drive/Colab Notebooks/Test/Data/base_video.mp4')   # your video dir

In [7]:
def display(img, video, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(video)):
        cols = [img]
        cols.append(video[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

In [8]:
# resize image and video
source_image = resize(source_image, (256, 256))[..., :3]
video_source = [resize(frame, (256, 256))[..., :3] for frame in video_source]

In [ ]:
# preview
HTML(display(source_image, video_source).to_html5_video())

In [11]:
output_name = 'final'

In [ ]:
generator, kp_detector = load_checkpoints(config_path='/content/first-order-model/config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/Colab Notebooks/Test/Data/vox-cpk.pth.tar')   # model dir

# create video 
predictions = make_animation(source_image, video_source, generator, kp_detector, relative=True)
imageio.mimsave(f'/content/gdrive/My Drive/Colab Notebooks/Test/Data/{output_name}.mp4', [img_as_ubyte(frame) for frame in predictions])   # image save dir

In [ ]:
# preview
HTML(display(source_image, video_source, predictions).to_html5_video())

In [ ]:
# extra features
# 1. Speed up video
# 2. Mix sound and video
# 3. Combine Video

In [ ]:
# Speed up video
# exsample 3.0 x speed up

!ffmpeg -i\
 '/content/gdrive/My Drive/Colab Notebooks/Test/Data/final.mp4' \        # input_dir
 -filter:v \
 "setpts=0.33333*PTS" \                                                  # speedup setting
 '/content/gdrive/My Drive/Colab Notebooks/Test/Data/final_speed_up.mp4' # output_dir

In [ ]:
# Mix sound and video

input_video = ffmpeg.input('/content/gdrive/My Drive/Colab Notebooks/Test/Data/final_speed_up.mp4')
input_audio = ffmpeg.input('/content/gdrive/My Drive/Colab Notebooks/Test/Data/sound.mp3')

ffmpeg.concat(input_video, input_audio, v=1, a=1).output('/content/gdrive/My Drive/Colab Notebooks/Test/Data/video_out.mp4').run()

In [ ]:
# 3. Combine Video

!ffmpeg -i\
 '/content/gdrive/My Drive/Colab Notebooks/Test/Data/video_out_1.mp4' -i\     # input 1
 '/content/gdrive/My Drive/Colab Notebooks/Test/Data/video_out_2.mp4' \       # input 2
 -filter_complex hstack \
 '/content/gdrive/My Drive/Colab Notebooks/Test/Data/video_out_combine.mp4'   # output dir video